In [1]:
# SET UP CONSTANTS
from  pathlib import Path

POP_DATA_PATH = Path('./data/ukpopulation2018.csv')
COVID_DATA_PATH = Path('./data/utla_2023-01-05.csv')
GEOJSON_PATH = Path(r'data/CTYUA_(Apr_2019)_Ultra_Generalised_Clipped_Boundaries_Great_Britain.geojson')
AREA_CODE = 'areaCode'
POP = 'pop'
DEATHS_COL = 'cum_deaths'
DEATH_100K = 'Deaths per 100k'
FORM_DATE ='Formatted Date'
MAP_TITLE = 'COVID 19 Deaths 28 Days After Positive Test, by Local Authority'

In [2]:
import geopandas as gpd
import pandas as pd
gdf = gpd.read_file(GEOJSON_PATH)
gdf.rename({'ctyua19cd': 'areaCode'}, axis=1, inplace=True)
gdf_reduced = gdf.drop(columns=[col for col in gdf.columns if col not in ['geometry', AREA_CODE]])

In [9]:
def reload_covid_data(time_compression='W', end_date=None) -> pd.DataFrame:
    covid_data = pd.read_csv(COVID_DATA_PATH, header=0)
    covid_data.rename({'cumDeaths28DaysByDeathDate': 'cum_deaths', 'cumPeopleVaccinatedFirstDoseByVaccinationDate': 'cum_vaccinated'}, inplace=True, axis=1)
    covid_data['date_sorter'] = pd.to_datetime(covid_data['date'])
    if end_date:
        covid_data = covid_data[covid_data['date_sorter'] <= end_date]
    covid_data[FORM_DATE] =  covid_data['date_sorter'].dt.strftime('%d/%m/%y')
    if time_compression in ['W', 'M']:
        covid_data = covid_data.groupby(AREA_CODE).resample(time_compression, on='date_sorter').min().droplevel(AREA_CODE).reset_index()
        covid_data = covid_data.sort_values(by='date_sorter').assign(date = covid_data['date_sorter'].dt.strftime('%d%m%Y')).drop(columns='date_sorter')
        covid_data= covid_data.reset_index(drop=True).fillna(0)
    else:    
        covid_data = covid_data.sort_values('date_sorter').drop(columns='date_sorter').reset_index(drop=True).fillna(0)
    
    return covid_data

In [4]:
def pop_data_clean(df):
    df = df.drop(columns=[col for col in df.columns if col not in ['Code', 'All Ages']]).rename(columns={'Code':AREA_CODE, 'All Ages': POP})
    print(df.columns)
    return df

In [5]:
import json
gdf_reduced.set_index('areaCode', inplace=True)
gdf_reduced_json = json.loads(gdf_reduced.to_json())

In [6]:
# join pop data to covid data 
covid_data = reload_covid_data()
pop_data = pop_data_clean(pd.read_csv(POP_DATA_PATH, header=0))
pop_data = pop_data.set_index(AREA_CODE)

covid_data = covid_data.join(pop_data, on=AREA_CODE, how='left')

Index(['areaCode', 'pop'], dtype='object')


In [10]:
covid_data = reload_covid_data(time_compression='W', end_date=pd.to_datetime('2021-03-01'))
pop_data = pop_data_clean(pd.read_csv(POP_DATA_PATH, header=0))
pop_data = pop_data.set_index(AREA_CODE)
covid_data = covid_data.join(pop_data, on=AREA_CODE, how='left')
covid_data[DEATH_100K] = (covid_data[DEATHS_COL] / covid_data[POP]) * 1E5

max_deaths = covid_data[DEATH_100K].max()

Index(['areaCode', 'pop'], dtype='object')


In [ ]:
import plotly.express as px
fig = px.choropleth_mapbox(data_frame=covid_data,
                           geojson=gdf_reduced_json,
                           locations=covid_data[AREA_CODE],
                           color=DEATH_100K,
                           # center={"lon": -1.88141, "lat": 52.484039},  # Brum
                           # featureidkey='properties.ctyua19cd',
                           zoom=5,
                           color_continuous_scale='blues',
                           range_color=(0, max_deaths),
                           animation_frame='date',
                           hover_name="areaName",
                           hover_data={'date': False, FORM_DATE:True,  POP: True, DEATH_100K: True, 'areaCode': False},
                           labels={POP: 'Population 2018',  FORM_DATE: 'Week beginning'},
                           width=800,
                           height=1300,
                           title=MAP_TITLE,
                          )
fig.update_layout(mapbox_style="carto-positron")
# fig.update_layout(margin={"r": 0, "t": 20, "l": 0, "b": 0})
fig.update_layout(mapbox_bounds={"west": -8, "east": 2, "south": 49, "north": 61})
fig.update_geos(fitbounds="locations")
fig_url = fig.write_html('covid_deaths_series.html')
fig.show()